# Anatomy of a multi-fluid model load

Peering into the innards of teqp

In [51]:
import timeit, json
import pandas
import numpy as np
import teqp

## Pure fluid loading

In [52]:
# By default teqp looks for fluids relative to the set of fluids in ROOT/dev/fluids
# The name (case-sensitive) should match the .json file, without the json extension.
%timeit model = teqp.build_multifluid_model(["Methane", "Ethane"], teqp.get_datapath())

9 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
# And if you provide valid aliases, alias lookup will be used to resolve the name
# But beware, this is rather a lot slower than the above because all fluid files need to be read
# in to build the alias map
%timeit model = teqp.build_multifluid_model(["n-C1H4", "n-C3H8"], teqp.get_datapath())

82.3 ms ± 431 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


So, how to make it faster? Only do it once and cache

In [54]:
# Here is the set of possible aliases to absolute paths of files
# Building this map takes a little while (somewhat faster in C++) due to all the file reads
# If you know your files will not change, good idea to build this alias map yourself.
%timeit aliasmap = teqp.build_alias_map(teqp.get_datapath())
type(aliasmap)

71.1 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


dict

In [55]:
# Then load the absolute paths from the alias map, 
# which will guarantee that you hit exactly what you were looking for,
# resolving aliases as needed
identifiers = [aliasmap[n] for n in ["Methane", "Ethane"]]
%timeit model = teqp.build_multifluid_model(identifiers, teqp.get_datapath())

8.93 ms ± 203 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


At some point soon teqp will support in-memory loading of JSON data for the pure components, without requiring reads from the operating system

## Mixture model loading

In [62]:
# Load the default JSON for the binary interaction parameters
BIP = json.load(open(teqp.get_datapath()+'/dev/mixtures/mixture_binary_pairs.json'))

In [63]:
# You can obtain interaction parameters either by pairs of names, where name is the name that teqp uses, the ["INFO"]["NAME"] field
params, swap_needed = teqp.get_BIPdep(BIP, ['Methane','Ethane'])
params

{'BibTeX': 'Kunz-JCED-2012',
 'CAS1': '74-82-8',
 'CAS2': '74-84-0',
 'F': 1.0,
 'Name1': 'Methane',
 'Name2': 'Ethane',
 'betaT': 0.996336508,
 'betaV': 0.997547866,
 'function': 'Methane-Ethane',
 'gammaT': 1.049707697,
 'gammaV': 1.006617867}

In [64]:
# Or also by CAS#
params, swap_needed = teqp.get_BIPdep(BIP, ['74-82-8','74-84-0'])
params

{'BibTeX': 'Kunz-JCED-2012',
 'CAS1': '74-82-8',
 'CAS2': '74-84-0',
 'F': 1.0,
 'Name1': 'Methane',
 'Name2': 'Ethane',
 'betaT': 0.996336508,
 'betaV': 0.997547866,
 'function': 'Methane-Ethane',
 'gammaT': 1.049707697,
 'gammaV': 1.006617867}

In [48]:
# But mixing is not allowed
params, swap_needed = teqp.get_BIPdep(BIP, ['74-82-8','Ethane'])
params

ValueError: Can't match the binary pair for: 74-82-8/Ethane